In [1]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
import utilities

In [3]:
from importlib import reload

In [4]:
reload(utilities)

<module 'utilities' from 'C:\\Users\\luisr\\SharedProjects\\HowToWinKaggle\\FinalProject\\utilities.py'>

In [5]:
sales = utilities.load_and_preprocess()
agg_sales = utilities.aggregate_monthly(sales)
piv_agg_sales = agg_sales.set_index(['Month', 'shop_id', 'item_id']).item_cnt_month.unstack(level=0).fillna(0)

In [6]:
%run test_utilities.py

..s.
----------------------------------------------------------------------
Ran 4 tests in 0.016s

OK (skipped=1)


In [7]:
df = piv_agg_sales

In [9]:
%%time
model = RandomForestRegressor(n_estimators=100, max_depth=4, random_state=42)
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 24, 1)

working on iteration: 0, oos month: 2015-02
working on iteration: 1, oos month: 2015-03
working on iteration: 2, oos month: 2015-04
working on iteration: 3, oos month: 2015-05
working on iteration: 4, oos month: 2015-06
working on iteration: 5, oos month: 2015-07
working on iteration: 6, oos month: 2015-08
working on iteration: 7, oos month: 2015-09
working on iteration: 8, oos month: 2015-10
Wall time: 4min 27s


In [10]:
errors

,in_sample,out_of_sample
oos_month,,
2015-02,1.687979,2.296264
2015-03,0.933817,1.207075
2015-04,1.046154,3.456593
2015-05,3.150708,2.293899
2015-06,1.891924,1.119179
2015-07,0.793567,0.838046
2015-08,0.671947,0.919195
2015-09,0.792254,4.574351
2015-10,2.745643,5.565774


In [11]:
import xgboost

In [12]:
%%time
model = xgboost.XGBRegressor(n_estimators=100, max_depth=4, random_state=42, objective='reg:squarederror')
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 24, 1)

working on iteration: 0, oos month: 2015-02
[16:26:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
working on iteration: 1, oos month: 2015-03
[16:26:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
working on iteration: 2, oos month: 2015-04
[16:26:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
working on iteration: 3, oos month: 2015-05
[16:27:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
working on iteration: 4, oos month: 2015-06
[16:27:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
working on iteration: 5, oos month: 2015-07
[16:27:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
working on iteration: 6, oos month: 2015

In [13]:
errors

,in_sample,out_of_sample
oos_month,,
2015-02,1.063748,2.925411
2015-03,0.818291,1.264791
2015-04,0.944669,3.466636
2015-05,3.108419,2.200026
2015-06,1.769666,1.394633
2015-07,0.680363,0.844092
2015-08,0.577643,0.863459
2015-09,0.660293,4.518485
2015-10,1.333799,7.018236


In [99]:
%%time
model = RandomForestRegressor(n_estimators=500, max_depth=4, random_state=42)
errors, _ = cross_val_score(piv_agg_sales, model, 24, 1)

working on iteration: 0, oos month: 2015-02
working on iteration: 1, oos month: 2015-03
working on iteration: 2, oos month: 2015-04
working on iteration: 3, oos month: 2015-05
working on iteration: 4, oos month: 2015-06
working on iteration: 5, oos month: 2015-07
working on iteration: 6, oos month: 2015-08
working on iteration: 7, oos month: 2015-09
working on iteration: 8, oos month: 2015-10
Wall time: 23min 24s


In [ ]:
errors

In [103]:
%%time
model = RandomForestRegressor(n_estimators=50, max_depth=8, random_state=42)
errors, _ = cross_val_score(piv_agg_sales, model, 24, 1)

working on iteration: 0, oos month: 2015-02
working on iteration: 1, oos month: 2015-03
working on iteration: 2, oos month: 2015-04
working on iteration: 3, oos month: 2015-05
working on iteration: 4, oos month: 2015-06
working on iteration: 5, oos month: 2015-07
working on iteration: 6, oos month: 2015-08
working on iteration: 7, oos month: 2015-09
working on iteration: 8, oos month: 2015-10
Wall time: 4min 27s


In [ ]:
errors

In [119]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = RandomForestRegressor(n_estimators=50, max_depth=4, random_state=42 )
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 24, 1, X_func_list=[clip])

working on iteration: 0, oos month: 2015-02
working on iteration: 1, oos month: 2015-03
working on iteration: 2, oos month: 2015-04
working on iteration: 3, oos month: 2015-05
working on iteration: 4, oos month: 2015-06
working on iteration: 5, oos month: 2015-07
working on iteration: 6, oos month: 2015-08
working on iteration: 7, oos month: 2015-09
working on iteration: 8, oos month: 2015-10
Wall time: 2min 18s


In [120]:
errors

,oos_month,in_sample,out_of_sample
0,2015-02,8.283748,5.038787
1,2015-03,1.192139,1.489250
2,2015-04,1.322192,12.028984
3,2015-05,10.587357,6.669743
4,2015-06,4.608624,2.164636
5,2015-07,1.169296,1.686021
6,2015-08,0.870168,1.590799
7,2015-09,1.043333,22.392229
8,2015-10,6.814748,19.907545


In [112]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = RandomForestRegressor(n_estimators=50, max_depth=4, random_state=42 )
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 24, 3, X_func_list=[clip])

working on iteration: 0, oos month: 2015-04
working on iteration: 1, oos month: 2015-05
working on iteration: 2, oos month: 2015-06
working on iteration: 3, oos month: 2015-07
working on iteration: 4, oos month: 2015-08
working on iteration: 5, oos month: 2015-09
working on iteration: 6, oos month: 2015-10
Wall time: 6min 48s


In [113]:
errors

,ise,oose
0,4.252698,11.985480
1,4.735913,6.132375
2,5.954689,1.916698
3,5.949740,1.640904
4,2.620008,1.514259
5,1.212551,22.546256
6,3.770698,15.721312


In [121]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = RandomForestRegressor(n_estimators=50, max_depth=4, random_state=42 )
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 24, 6, X_func_list=[clip])

working on iteration: 0, oos month: 2015-07
working on iteration: 1, oos month: 2015-08
working on iteration: 2, oos month: 2015-09
working on iteration: 3, oos month: 2015-10
Wall time: 10min 40s


In [122]:
errors

,oos_month,in_sample,out_of_sample
0,2015-07,5.400832,1.467589
1,2015-08,3.787161,1.515721
2,2015-09,3.793601,22.525362
3,2015-10,6.453089,14.676749


In [130]:
errors.set_index('oos_month').apply(np.sqrt)

,in_sample,out_of_sample
oos_month,,
2015-07,2.323969,1.211441
2015-08,1.946063,1.231146
2015-09,1.947717,4.746089
2015-10,2.540293,3.831024


In [147]:
errors.reset_index()

,index,oos_month,in_sample,out_of_sample
0,0,2015-07,2.592260,1.299016
1,1,2015-08,2.569755,1.292639
2,2,2015-09,2.544163,4.736078
3,3,2015-10,2.750446,3.722276


In [141]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = RandomForestRegressor(n_estimators=50, max_depth=4, random_state=42 )
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 18, 12, X_func_list=[clip])

working on iteration: 0, oos month: 2015-07
working on iteration: 1, oos month: 2015-08
working on iteration: 2, oos month: 2015-09
working on iteration: 3, oos month: 2015-10
Wall time: 14min 35s


In [142]:
errors

,oos_month,in_sample,out_of_sample
0,2015-07,2.592260,1.299016
1,2015-08,2.569755,1.292639
2,2015-09,2.544163,4.736078
3,2015-10,2.750446,3.722276


In [184]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = RandomForestRegressor(n_estimators=50, max_depth=4, random_state=42 )
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 24, 1, add_pred_month=True, X_func_list=[clip])

working on iteration: 0, oos month: 2015-02
working on iteration: 1, oos month: 2015-03
working on iteration: 2, oos month: 2015-04
working on iteration: 3, oos month: 2015-05
working on iteration: 4, oos month: 2015-06
working on iteration: 5, oos month: 2015-07
working on iteration: 6, oos month: 2015-08
working on iteration: 7, oos month: 2015-09
working on iteration: 8, oos month: 2015-10
Wall time: 2min 17s


In [186]:
errors

,in_sample,out_of_sample
oos_month,,
2015-02,8.285843,5.056702
2015-03,1.190970,1.494261
2015-04,1.322160,12.027061
2015-05,10.585138,6.672631
2015-06,4.609989,2.192537
2015-07,1.168365,1.701230
2015-08,0.872613,1.623951
2015-09,1.048895,22.413778
2015-10,6.854944,19.698577


In [187]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = RandomForestRegressor(n_estimators=50, max_depth=4, random_state=42 )
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 18, 12, add_pred_month=True, X_func_list=[clip])

working on iteration: 0, oos month: 2015-07
working on iteration: 1, oos month: 2015-08
working on iteration: 2, oos month: 2015-09
working on iteration: 3, oos month: 2015-10
Wall time: 15min 26s


In [188]:
errors

,in_sample,out_of_sample
oos_month,,
2015-07,2.546802,1.239207
2015-08,2.525237,1.261689
2015-09,2.497281,4.734698
2015-10,2.698522,3.729362


This takes waaaay too long, had to interrupt

In [189]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = RandomForestRegressor(n_estimators=1000, max_depth=8, random_state=42 )
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 18, 12, add_pred_month=True, X_func_list=[clip])

working on iteration: 0, oos month: 2015-07


KeyboardInterrupt: 

In [14]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = xgboost.XGBRegressor(n_estimators=500, max_depth=5, random_state=42, objective='reg:squarederror')
errors, _ = utilities.cross_val_score(piv_agg_sales.iloc[:,3:], model, 18, 12, add_pred_month=True, X_func_list=[clip])

working on iteration: 0, oos month: 2015-10
[16:34:03] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
Wall time: 48min 16s


In [15]:
errors

,in_sample,out_of_sample
oos_month,,
2015-10,2.127563,3.818562


In [ ]:
%%time
def clip(X):
    return np.clip(X, 0, 20)

model = RandomForestRegressor(n_estimators=500, max_depth=5, random_state=42 )
errors, _ = utilities.cross_val_score(piv_agg_sales, model, 18, 12, add_pred_month=True, X_func_list=[clip])

working on iteration: 0, oos month: 2015-07
working on iteration: 1, oos month: 2015-08
working on iteration: 2, oos month: 2015-09


In [ ]:
errors

## TODO
* create score from rolling fits
* add shop 'bouginess' score
* add which month is this feature